In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("/home/bnet/litalv/MLHC/model")

In [3]:
import pandas as pd
import numpy as np
import pickle as pkl

from model.trainning import train_multitask_seq_ae
from model.prediction import predict_proba, encode_embeddings, predict_proba_knn, predict_logits, fit_calibrators, predict_proba_calibrated,fit_isotonic_calibrators, predict_isotonic_proba_calibrated
from model.utils.eval_results import eval_multitask_from_probs, plot_running_total_subplots, plot_running_total, plot_calibration_curve
from model.utils.feature_importance import feature_importance

# Read dataSets

In [4]:
with open(f"data/X_train_seq.pkl", 'rb') as f:
	X_train = pkl.load(f)
with open(f"data/y_train_lables.pkl", 'rb') as f:
	y_train = pkl.load(f)
with open(f"data/mask_train.pkl", 'rb') as f:
	mask_train = pkl.load(f)

with open(f"data/X_test_seq.pkl", 'rb') as f:
	X_test = pkl.load(f)
with open(f"data/y_test_lables.pkl", 'rb') as f:
	y_test = pkl.load(f)
with open(f"data/mask_test.pkl", 'rb') as f:
	mask_test = pkl.load(f)

with open(f"data/X_val_seq.pkl", 'rb') as f:
	X_val = pkl.load(f)
with open(f"data/y_val_lables.pkl", 'rb') as f:
	y_val = pkl.load(f)
with open(f"data/mask_val.pkl", 'rb') as f:
	mask_val = pkl.load(f)

with open(f"data/models_params_dict.pkl", 'rb') as f:
	models_params_dict = pkl.load(f)
TASKS = models_params_dict['tasks']

# Train model

#### def targets weights using the training set

In [5]:
train_N_pos = y_train.sum(axis=0)
train_N_neg = y_train.shape[0] - y_train.sum(axis=0)
beta = 0.999

weights_bce = train_N_neg / train_N_pos
weights_supcon = (1.0 - beta) / (1.0 - (beta ** train_N_pos))

#### running model

In [6]:
model, losses = train_multitask_seq_ae(
    X_train, y_train, mask_train,
    input_dim=X_train.shape[-1],
    batch_size=64,
    p_per_task=6, # ensure ≥6 positives per task per batch
    epochs=20,
    scale_warmup_epochs=1,
    lr_warmup_epochs=0,

    warmup_scales={'lambda_recon':1.0, 'lambda_bce':0.0, 'lambda_supcon':1.0},
    scales={'lambda_recon':0.2, 'lambda_bce':1.0, 'lambda_supcon':2.0},

    warmup_lrs={'AE':0, 'BCE':[0, 0, 0], 'SupCon':0},
    lrs={'AE':1e-3, 'BCE':[1e-3, 1e-3, 1e-4], 'SupCon':1e-3},

    latent_dim=64, 
    SupCon_latent_dim=32,

    pooling_mode ="mean+max+final", # "final", "mean+final", "mean+max+final", "mean+attn"

    weights_bce=weights_bce,
    weights_supcon=weights_supcon,
    temperature=[0.07,0.07], 
    supcon_gamma=0.7,
    supcon_delta=0.5,
    seed=42
)
# plot_running_total(losses) 

probs_val = predict_proba(model, X_val, mask_val)  
probs_test = predict_proba(model, X_test, mask_test)  
r_val = eval_multitask_from_probs(y_val, probs_val, task_names = TASKS, plot=False)
r_test = eval_multitask_from_probs(y_test, probs_test, task_names = TASKS, plot=False)

print(i)
print("BEC val prediction")
for i in r_val:
    print(i, r_val[i]['roc_auc'], r_val[i]['pr_auc'])
print("BEC test prediction")
for i in r_test:
    print(i, r_test[i]['roc_auc'], r_test[i]['pr_auc'])

NameError: name 'i' is not defined

# Full Fine-tuneinig

In [33]:
# tune_optuna_min.py
from typing import Dict, List, Any
import numpy as np
import optuna
from optuna.samplers import TPESampler

# ---------- metric selector (uses your r_val structure) ----------
def make_scorer(metric: str, task_idx=None, tasks=None):
    """
    metric: 'roc' | 'ap'
    task_idx: None -> mean over 3 tasks, else 0/1/2 for a single task.
    tasks: list of task names (e.g., ["mortality","prolonged_stay","readmission"])
    """
    key = "roc_auc" if metric == "roc" else "pr_auc"

    def score_fn(r_val):
        # r_val may be a dict keyed by names OR a list indexed by 0..2
        if isinstance(r_val, dict):
            # dict keyed by task names
            if task_idx is None:
                names = tasks or list(r_val.keys())
                return float(np.mean([r_val[name][key] for name in names]))
            else:
                name = (tasks[task_idx] if tasks else task_idx)
                return float(r_val[name][key])
        else:
            # list/tuple indexed by 0..2
            if task_idx is None:
                return float(np.mean([r_val[i][key] for i in range(3)]))
            else:
                return float(r_val[task_idx][key])
    return score_fn

# ---------- one train→eval using your API ----------
def run_once(params, data, epochs_for_tuning = 12, seed = 42):
    model, _ = train_multitask_seq_ae(
        data["X_train"], data["y_train"], data["mask_train"],
        input_dim=data["X_train"].shape[-1],

        batch_size=params["batch_size"],
        p_per_task=params["p_per_task"],
        epochs=epochs_for_tuning,

        warmup_epochs=params["warmup_epochs"],
        warmup_scales=params["warmup_scales"],

        max_lr=params["max_lr"],
        min_lr=params["min_lr"],

        latent_dim=params["latent_dim"],
        SupCon_latent_dim=params["SupCon_latent_dim"],
        pooling_mode=params["pooling_mode"],

        lambda_recon=params["lambda_recon"],
        lambda_bce=params["lambda_bce"],
        lambda_supcon=params["lambda_supcon"],

        weights_bce=data["weights_bce"],
        weights_supcon=data["weights_supcon"],

        temperature=params["temperature"],
        supcon_gamma=params["supcon_gamma"],
        supcon_delta=params["supcon_delta"],
        seed=seed,
    )
    probs_val = predict_proba(model, data["X_val"], data["mask_val"])
    r_val = eval_multitask_from_probs(data["y_val"], probs_val, task_names=data["TASKS"], plot=False)
    return r_val

# ---------- minimal Optuna runner ----------
def optuna_search(
    data: Dict[str, Any],
    base_params: Dict[str, Any],
    spaces: Dict[str, List[Any]],   # discrete candidates per param (categorical only)
    metric: str = "roc",            # 'roc' | 'ap'
    task_idx = None,    # 0/1/2 for a single task; None = mean across tasks
    n_trials: int = 20,
    epochs_for_tuning: int = 12,
    seed: int = 42,
):
    """ choose *exact* values to test in `spaces`. 
        anything not listed uses `base_params`. """
    scorer = make_scorer(metric, task_idx, tasks=data.get("TASKS"))
    study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=seed))

    def objective(trial: optuna.Trial):
        params = dict(base_params)
        for name, candidates in spaces.items():
            idx = trial.suggest_categorical(f"{name}_idx", list(range(len(candidates))))
            params[name] = candidates[idx]
        r_val = run_once(params, data, epochs_for_tuning=epochs_for_tuning, seed=seed + trial.number)
        return scorer(r_val)

    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    # Rebuild best params (resolve *_idx → actual value)
    best_params = dict(base_params)
    for name, candidates in spaces.items():
        best_idx = study.best_trial.params[f"{name}_idx"]
        best_params[name] = candidates[int(best_idx)]

    # Short confirm run (optional)
    r_val_best = run_once(best_params, data, epochs_for_tuning=epochs_for_tuning, seed=seed)
    print("\nBest score:", scorer(r_val_best))
    print("Best params:", {k: best_params[k] for k in sorted(best_params)})
    print("Best r_val:", r_val_best)
    return best_params, study

In [ ]:
data = dict(
    X_train=X_train, y_train=y_train, mask_train=mask_train,
    X_val=X_val,     y_val=y_val,     mask_val=mask_val,
    weights_bce=weights_bce, weights_supcon=weights_supcon,
    TASKS=TASKS,
)

# Baseline (anything not varied in 'spaces' stays as-is)
base = dict(
    batch_size=96,
    p_per_task=4,
    warmup_epochs=0,
    warmup_scales=(0.0, 0.0, 0.0),

    max_lr=5e-4,
    min_lr=1e-4,

    latent_dim=80,
    SupCon_latent_dim=16,
    pooling_mode="mean+final",

    lambda_recon=0.3,
    lambda_bce=1.0,
    lambda_supcon=1.0,

    temperature=0.0953,
    supcon_gamma=0.7,
    supcon_delta=0.5,
)

# Choose exactly what to vary (add/remove freely)
spaces = {
    # batching / sampling
    "batch_size": [64, 96, 128],
    "p_per_task": [4, 6, 8, 10],

    # warmup & LR
    "warmup_epochs": [0, 1, 2],
    "warmup_scales": [(0.0, 0.0, 1.0), (0.0, 0.5, 1.0), (0.5, 0.0, 2.0)],
    "max_lr": [5e-4, 7.5e-4, 1e-3],
    "min_lr": [5e-5, 1e-4, 2e-4],

    # capacity / pooling
    "latent_dim": [64, 80, 96, 128],
    "SupCon_latent_dim": [16, 32, 48, 64],
    "pooling_mode": ["mean+final", "max+final", "mean+max+final"],

    # losses & SupCon
    "lambda_recon": [0.1, 0.2, 0.3],
    "lambda_bce": [0.8, 1.0, 1.2],
    "lambda_supcon": [1.0, 2.0, 3.0],
    "temperature": [0.06, 0.08, 0.10, 0.12],
    # "supcon_gamma": [0.6, 0.7, 0.8],
    # "supcon_delta": [0.4, 0.5, 0.6],
}

# Optimize MEAN AP across 3 tasks
best_mean_ap, study_mean_ap = optuna_search(
    data, base, spaces,
    metric="ap", task_idx=None,
    n_trials=200, epochs_for_tuning=12, seed=42
)

best_task_roc, study_task_roc, best_task_ap, study_task_ap = [],[],[],[]
for i in range(3):
    # Optimize ROC for a single task (e.g., readmission = task 2)
    best_task2_roc, study_task2_roc = optuna_search(
        data, base, spaces,
        metric="roc", task_idx=i,
        n_trials=24, epochs_for_tuning=12, seed=42
    )
    best_task_roc.append(best_task2_roc)
    study_task_roc.append(study_task2_roc)

    # Optimize AP for the same task
    best_task2_ap, study_task2_ap = optuna_search(
        data, base, spaces,
        metric="ap", task_idx=i,
        n_trials=24, epochs_for_tuning=12, seed=42
    )
    best_task_ap.append(best_task2_ap)
    study_task_ap.append(study_task2_ap)

# Weights scaling fine-tuning 

In [ ]:
# --- Objective to maximize ---
def objective(trial):
    data = dict(
        X_train=X_train, y_train=y_train, mask_train=mask_train,
        X_val=X_val,     y_val=y_val,     mask_val=mask_val,
        weights_bce=weights_bce, weights_supcon=weights_supcon,
        TASKS=TASKS,
    )
    
    # Suggest hyperparameters
    supcon_gamma          = trial.suggest_float("supcon_gamma", 0.2, 0.7)
    supcon_delta       = trial.suggest_float("supcon_delta", 0.2, 0.7, log=True)

    # Train with suggested params
    model, _ = train_multitask_seq_ae(
        data["X_train"], data["y_train"], data["mask_train"],
        input_dim=data["X_train"].shape[-1],

        batch_size=64,
        p_per_task=6,
        epochs=20,

        warmup_epochs=0,
        warmup_scales=(0.0, 0.0, 1.0),

        max_lr=5e-4,
        min_lr=1e-4,

        latent_dim=80,
        SupCon_latent_dim=16,
        pooling_mode="mean+final",

        lambda_recon=0.3,
        lambda_bce=1,
        lambda_supcon=1,

        weights_bce=data["weights_bce"],
        weights_supcon=data["weights_supcon"],

        temperature=[0.12,0.095],
        supcon_gamma=supcon_gamma,
        supcon_delta=supcon_delta,
        seed=42,
    )

    # Evaluate on validation using ROC-AUC (mean of available tasks)
    probs_val = predict_proba(model, X_val, mask_val)   # shape: (N, 3)
    r_val = eval_multitask_from_probs(y_val, probs_val, task_names = TASKS, plot=False)
    mean_auc = float(np.mean([r_val[i]['roc_auc'] for i in r_val]))

    return mean_auc  # Optuna will MAXIMIZE this

# --- Run the search ---
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, n_jobs=1)  # increase n_trials as you like

print("Best AUC:", study.best_value)
print("Best params:", study.best_params)

In [ ]:
print("Best AUC:", study.best_value)
print("Best params:", study.best_params)

# compare configurations

In [48]:
data = dict(
    X_train=X_train, y_train=y_train, mask_train=mask_train,
    X_val=X_val,     y_val=y_val,     mask_val=mask_val,
    weights_bce=weights_bce, weights_supcon=weights_supcon,
    TASKS=TASKS,
)

# ===== Suggested configs =====
config_names = [ "baseline",
  "M1_lambdaSupCon_0.8",
  "M2_temp_0.10",
  "M3_supconLatent_32",
  "M4_pPerTask_6",
  "M5_longer_train_lower_minLR",
  "M6_light_warmup_bce_first"
]

baseline_config = dict(
    batch_size=96,
    p_per_task=4,
    warmup_epochs=0,
    warmup_scales=(0.0, 0.0, 1.0),

    max_lr=5e-4,
    min_lr=1e-4,

    latent_dim=80,
    SupCon_latent_dim=16,
    pooling_mode="mean+final",

    lambda_recon=0.3,
    lambda_bce=1.0,
    lambda_supcon=1.0,

    temperature=0.09533360660422997,
    supcon_gamma=0.7,
    supcon_delta=0.5,
)

configs = [
  # M1: Reduce SupCon dominance slightly
  dict(batch_size=96, p_per_task=4, warmup_epochs=0, warmup_scales=(0.0,0.0,1.0),
       max_lr=5e-4, min_lr=1e-4,
       latent_dim=80, SupCon_latent_dim=16, pooling_mode="mean+final",
       lambda_recon=0.3, lambda_bce=1.0, lambda_supcon=0.8,
       temperature=0.0953, supcon_gamma=0.7, supcon_delta=0.5),

  # M2: Softer SupCon via temperature
  dict(batch_size=96, p_per_task=4, warmup_epochs=0, warmup_scales=(0.0,0.0,1.0),
       max_lr=5e-4, min_lr=1e-4,
       latent_dim=80, SupCon_latent_dim=16, pooling_mode="mean+final",
       lambda_recon=0.3, lambda_bce=1.0, lambda_supcon=1.0,
       temperature=0.10, supcon_gamma=0.7, supcon_delta=0.5),

  # M3: Slightly larger projection (can help AP without cranking λ_supcon)
  dict(batch_size=96, p_per_task=4, warmup_epochs=0, warmup_scales=(0.0,0.0,1.0),
       max_lr=5e-4, min_lr=1e-4,
       latent_dim=80, SupCon_latent_dim=32, pooling_mode="mean+final",
       lambda_recon=0.3, lambda_bce=1.0, lambda_supcon=1.0,
       temperature=0.0953, supcon_gamma=0.7, supcon_delta=0.5),

  # M4: A touch more positives per task (you saw 8–10 hurt; try just 6)
  dict(batch_size=96, p_per_task=6, warmup_epochs=0, warmup_scales=(0.0,0.0,1.0),
       max_lr=5e-4, min_lr=1e-4,
       latent_dim=80, SupCon_latent_dim=16, pooling_mode="mean+final",
       lambda_recon=0.3, lambda_bce=1.0, lambda_supcon=1.0,
       temperature=0.0953, supcon_gamma=0.7, supcon_delta=0.5),

  # M5: Let BCE descend more — longer run & smaller tail LR
  dict(batch_size=96, p_per_task=4, warmup_epochs=0, warmup_scales=(0.0,0.0,1.0),
       max_lr=5e-4, min_lr=5e-5,  # ↓ tail LR
       latent_dim=80, SupCon_latent_dim=16, pooling_mode="mean+final",
       lambda_recon=0.3, lambda_bce=1.0, lambda_supcon=1.0,
       temperature=0.0953, supcon_gamma=0.7, supcon_delta=0.5,
       # run this one for 25–30 epochs in your loop
  ),

  # M6: Give BCE a head start (very light warmup)
  dict(batch_size=96, p_per_task=4, warmup_epochs=1, warmup_scales=(0.0,0.5,1.0),
       max_lr=5e-4, min_lr=1e-4,
       latent_dim=80, SupCon_latent_dim=16, pooling_mode="mean+final",
       lambda_recon=0.3, lambda_bce=1.0, lambda_supcon=1.0,
       temperature=0.0953, supcon_gamma=0.7, supcon_delta=0.5),
]

for c, conf in enumerate([baseline_config]+configs):
    print(f"\n=== {c}: {config_names[c]} ===")
    model, _ = train_multitask_seq_ae(
        data["X_train"], data["y_train"], data["mask_train"],
        input_dim=data["X_train"].shape[-1],

        batch_size=conf["batch_size"],
        p_per_task=conf["p_per_task"],
        epochs=20,

        warmup_epochs=conf["warmup_epochs"],
        warmup_scales=conf["warmup_scales"],

        max_lr=conf["max_lr"],
        min_lr=conf["min_lr"],

        latent_dim=conf["latent_dim"],
        SupCon_latent_dim=conf["SupCon_latent_dim"],
        pooling_mode=conf["pooling_mode"],

        lambda_recon=conf["lambda_recon"],
        lambda_bce=conf["lambda_bce"],
        lambda_supcon=conf["lambda_supcon"],

        weights_bce=data["weights_bce"],
        weights_supcon=data["weights_supcon"],

        temperature=conf["temperature"],
        supcon_gamma=conf["supcon_gamma"],
        supcon_delta=conf["supcon_delta"],
        seed=42,
    )
    
    probs_val = predict_proba(model, X_val, mask_val)  
    probs_test = predict_proba(model, X_test, mask_test)  
    r_val = eval_multitask_from_probs(y_val, probs_val, task_names = TASKS, plot=False)
    r_test = eval_multitask_from_probs(y_test, probs_test, task_names = TASKS, plot=False)

    print("BEC val prediction")
    for i in r_val:
        print(i, r_val[i]['roc_auc'], r_val[i]['pr_auc'])
    print()

# Eval each models part contribution

In [52]:
import copy

BASE_ARGS = dict(
    batch_size=96,
    p_per_task=4,
    warmup_epochs=0,
    warmup_scales=(0.0, 0.0, 1.0),

    max_lr=5e-4,
    min_lr=1e-4,

    latent_dim=80,
    SupCon_latent_dim=16,
    pooling_mode="mean+final",

    lambda_recon=0.3,
    lambda_bce=1.0,
    lambda_supcon=1.0,

    temperature=0.09533360660422997,
    supcon_gamma=0.7,
    supcon_delta=0.5,
    seed=42
)

ABLATIONS = {
    # baseline (all losses on)
    "baseline": dict(lambda_recon=0.2, lambda_bce=1.0, lambda_supcon=1.0, warmup_epochs=0),
    # remove SupCon term only
    "no_supcon": dict(lambda_recon=0.2, lambda_bce=1.0, lambda_supcon=0.0, warmup_epochs=0),
    # remove Recon term only
    "no_recon": dict(lambda_recon=0.0, lambda_bce=1.0, lambda_supcon=1.0, warmup_epochs=0),
    # BCE only (both Recon and SupCon off)
    "bce_only": dict(lambda_recon=0.0, lambda_bce=1.0, lambda_supcon=0.0, warmup_epochs=0),
    # SupCon only (heads won’t train; use KNN for eval)
    "supcon_only": dict(lambda_recon=0.0, lambda_bce=0.0, lambda_supcon=1.0, warmup_epochs=0),
}

def run_one(name, xtr, ytr, mtr, xte, yte, mte):
    # merge base args + ablation-specific overrides
    args = copy.deepcopy(BASE_ARGS)
    args.update(ABLATIONS[name])
    # train
    model, _ = train_multitask_seq_ae(
        xtr, ytr, mtr,
        input_dim=xtr.shape[-1],
        weights_bce=weights_bce,
        weights_supcon=weights_supcon,
        **args
    )
    # eval
    out = {"setting": name}

    # BCE probabilities (only meaningful if lambda_bce>0)
    if args["lambda_bce"] > 0.0:
        probs_bce = predict_proba(model, xte, mte)
        rep_bce = eval_multitask_from_probs(yte, probs_bce, plot=False, task_names=TASKS)
        for t in TASKS:
            out[f"{t}_ROC_bce"] = rep_bce[t]["roc_auc"]
            out[f"{t}_PR_bce"]  = rep_bce[t]["pr_auc"]
    else:
        for t in TASKS:
            out[f"{t}_ROC_bce"] = None
            out[f"{t}_PR_bce"]  = None

    # KNN over embeddings (meaningful for all; *primary* for supcon_only)
    probs_knn = predict_proba_knn(model, xtr, xte, mtr, mte, ytr, n_neig=10)
    rep_knn = eval_multitask_from_probs(yte, probs_knn, plot=False, task_names=TASKS)
    for t in TASKS:
        out[f"{t}_ROC_knn"] = rep_knn[t]["roc_auc"]
        out[f"{t}_PR_knn"]  = rep_knn[t]["pr_auc"]

    return out

# ==== run all ====
rows = []
for name in ABLATIONS:
    rows.append(run_one(name, X_train, y_train, mask_train, X_test, y_test, mask_test))
results_df = pd.DataFrame(rows)

# pretty order
cols = ["setting"]
for src in ["bce","knn"]:
    for t in TASKS:
        cols += [f"{t}_ROC_{src}", f"{t}_PR_{src}"]
results_df = results_df[cols]

print(results_df.to_string(index=False))

#### Save model

In [7]:
import torch
torch.save(model, "model.pth")